# Clustering

TODO:

* Adjusted mutual information and silhouette
* Choosing number of clusters via elbow method
* Doing dimensionality reduction to avoid curse of dimensionality

## Clustering as unsupervised learning

## The problem: clustering gene expression data

Trapnell (2014) performed single-cell RNA-seq on differentiating skeletal myoblasts at 0, 24, 48, and 72 hours. The original data are available via [GEO database accession number GSE52529](ftp://ftp.ncbi.nlm.nih.gov/geo/series/GSE52nnn/GSE52529/suppl/GSE52529_fpkm matrix.txt.gz). We have at hand a cleaned-up version from [Sincell](https://www.bioconductor.org/packages/3.3/bioc/vignettes/sincell/inst/doc/sincell-vignette.pdf), in which only the expression levels for 575 differentially-expressed genes have been retained across 271 cells drawn from the various timepoints.

Our goal is to cluster the 271 cells based on gene expression. *Clustering* is simply a means of grouping related items in your data together, allowing you to gain insight into overarching trends. In this case, we expect cells at each timepoint to have relatively similar expression levels across different genes, meaning that we hope cells from the same timepoints will cluster together. In this regard, knowing what timepoint each cell comes from provides us with a "ground truth" means of evaluating our clustering&mdash;if we see cells from the same timepoint present in the same cluster, it means our clustering is working as intended. This will give us confidence in applying our clustering methods to other single-cell datasets, where we hope to discover underlying structure based on factors other than time since cell division.

## How k-means clustering works

K-means is an extremely simple clustering algorithm that is nonetheless quite effective.

1. Decide how many clusters $k$ you wish to use.
2. Place $k$ cluster centres ("centroids") at random locations in the space in which your data lives.
3. Assign each data point to the cluster corresponding to its closest centroid.
4. Repeat:
    1. For each cluster, move its centroid to the centre (i.e., mean) of all points belonging to the cluster.
    2. Assign each data point to its closest centroid.
    3. If no points changed their cluster assignments, exit. Otherwise, return to step 4.

The following example illustrates the algorithm. Cluster centroids are circles, while data points are squares. Here, we have arbitrarily chosen to create $k=3$ clusters.

| Step | Illustration |
| ---- | ------------ |
| Step 1: Place $k=3$ centroids randomly. Here, we have arbitrariliy chosen $k=3$&mdash;we could have easily used a different value of $k$. | ![K-means step 1](images/kmeans_example_step1.svg) |
| Step 2: Assign each data point to its closest centroid. | ![K-means step 2](images/kmeans_example_step2.svg) |
| Step 3: Move each centroid to the centre of all data points belonging to its cluster. | ![K-means step 3](images/kmeans_example_step3.svg) |
| Step 4:Reassign each data point to the cluster with the closest centroid. | ![K-means step 4](images/kmeans_example_step4.svg) 

We repeat these steps until convergence&mdash;that is, when no points change cluster assignments. Note that the precise solution you obtain will depend on the random locations you initially placed your centroids. Rerunning the algorithm with different starting points for the centroids may change the cluster assignments of some points.

## How to fit a mixture of Gaussians using expectation maximization

As an alternative to k-means, we can fit a *mixture of Gaussians* using *expectation maximization*. Let's first discuss what a mixture of Gaussians is. Imagine you have a bunch of points that you know were sampled from different Gaussian probability distributions.

![MOG truth](images/mog_truth.png)

Here, you see points drawn from three different Gaussians. Each Gaussian has parameters describing its mean and variance, which determine the position and shape of the distribution. As there is more probability mass under the "centre" of the Guassians than under the "tails", we see more points sampled under the centre of each distribution. But, of course, if you were to encounter this data in the real world, you wouldn't know the parameters of the Gaussians, so you would have no idea what the distributions look like.

![MOG truth](images/mog_no_pdfs.png)

In fact, you don't even know which Gaussian each point was sampled from.

![MOG truth](images/mog_no_labels.png)

However, to reduce the complexity of the problem, let's assume for the moment that you do know that the data was drawn from Gaussian distributions, however, and that there were originally three Gaussians. Given this knowledge, how do you determine the parameters for each Gaussian, and consequently what distribution each point belongs to? The answer is expectation maximization.

Expectation maximization is an iterative algorithm that lets you establish both the parameters of the distributions from which your data was generated, as well as the most likely assignments of points to distributions. As such, in our example, we can think of each Gaussian corresponding to a cluster. To determine the cluster parameters and assignments, we use the following algorithm:

1. E (expectation) step: Each cluster has associated with it parameters describing its properties. For our Gaussian example, this corresponds to a mean and variance for each Gaussian describing the position and shape of the distribution. For each data point, determine the *likelihood* that it was generated by each cluster.

2. M (maximization) step: For each data point and each cluster, update the parameters of the cluster based on the points assigned to it.

3. If the cluster assignments and parameters hardly changed (i.e., converged) in the E and M steps, exit. Otherwise return to step 1.

![MOG all steps](images/mog_all_steps.png)

Here we observe this process in action.

* In step 1, we initialize three Gaussians with random means and variances.
* In step 2, we have performed our first E and M steps. Each point is associated with its most likely Gaussian (E step); each Gaussian is adjusted to better fits its associated points (M step).
* In step 3, we perform another run of the E and M steps.
* In step 4, the algorithm has converged and so we terminate. The answer we arrive at precisely matches the ground truth.

What is not captured by the image is that, unlike k-means, expectation maximization generates *soft* rather than *hard* assignments of points to clusters. K-means' assignments are "hard" because, at each step, every point is assigned with certainty to one and only one cluster. Expectation maximization, conversely, generates "soft" assignments, as every point is assigned with some probability to every cluster. This has two consequences. Firstly, the higher the probability of a point's assignment to a given cluster, the more "weight" that point will have in determining that cluster's parameters. For example, in step 2 of our EM example, the points directly under the centre of the red Gaussian will have a large influence in determining the updated mean of the distribution for step 3. But even the green points on the far right of the axis will have a small influence on the new mean of the red Gaussian, in accordance with their infinitesimal probability of having been generated by the red Gaussian. Once the algorithm converges, we can get hard assignments of points to clusters by simply taking the most probable cluster for each point.

Relative to k-means, expectation maximization is a more general algorithm that can be used for purposes other than clustering. In fact, EM can be applied to a wide class of problems in which we confront *latent* (hidden) variables that affect our inference. In this example, the latent variables (or hidden knowledge) are which distribution generated each point. But you can also apply EM to diverse other problems. Imagine you were flipping two coins, each with some unknown bias towards or tails. Given the results of 100 coin flips (heads, tails, heads, heads, ...), in which one coin or the other was selected randomly at each step, you want to know which coin was flipped at each step, and what bias each coin has. This problem appears quite difficult, as you lack knowledge not only of the bias parameter associated with each coin, but also the identity of the coin used for each flip. Expectation maximization gives you a principled way of determining the most likely solution to this problem.

## Simulated clustering

In [4]:
from cluster import run_simulated
run_simulated()

[[-3.40668197  1.68653758]
 [ 1.56975907  1.59487466]
 [-4.93425895  4.56220774]
 [ 2.36019663  1.14894076]
 [-3.84745114  2.14805512]
 [ 3.5661392   1.42438541]
 [-0.69261162 -3.94876848]
 [-2.37621142 -5.3188365 ]
 [-3.87588566  1.93759412]
 [-4.39707614  3.84360606]
 [-1.90152152 -4.80069395]
 [ 2.36127428  1.95124545]
 [-5.57689135  2.91115443]
 [-6.03912866  4.2594258 ]
 [-1.7897726  -4.24561122]
 [ 3.205313    2.70060585]
 [ 3.80073222  2.67239015]
 [-2.40567392 -4.6089913 ]
 [ 2.19428562  1.80213024]
 [-4.43089198  3.9666052 ]
 [ 3.98833067  2.01947583]
 [-1.49338779 -4.02730121]
 [-3.17080414 -5.39216147]
 [-3.42879913  3.42666611]
 [-6.26750821  2.75198668]
 [ 3.55078108  3.26553442]
 [-2.70362091 -5.97072686]
 [ 2.65859808 -0.02724194]
 [-3.8226584  -3.78325589]
 [-3.284733   -4.03736813]] [ 0.  2.  0.  2.  0.  2.  1.  1.  0.  0.  1.  2.  0.  0.  1.  2.  2.  1.
  2.  0.  2.  1.  1.  0.  0.  2.  1.  2.  1.  1.]


## Preparing to cluster our gene expression data

We have a 271x575 matrix containing gene expression levels. Each row corresponds to a cell, while each column corresponds to the expression of a gene in $\log\text{FPKM}$ units. We will perform dimensionality reduction via PCA on our data before clustering, for reasons that we will discuss later.

In [1]:
import numpy as np
from cluster import get_timepoints, plot
import sklearn.decomposition

def load_exprmat(exprmat_fn):
  rows = []
  rownames = []

  with open(exprmat_fn) as exprmat:
    colnames = next(exprmat).strip().split(',')
    for row in exprmat:
      fields = row.split(',')
      rownames.append(fields[0])
      rows.append([float(f) for f in fields[1:]])

  data = np.array(rows)
  return (data, colnames, rownames)

def reduce_dimensionality(exprmat, n_components):
  pca = sklearn.decomposition.PCA(n_components=n_components)
  projected = pca.fit(exprmat).transform(exprmat)
  print('Explained variance ratio: %s' % pca.explained_variance_ratio_)
  return projected

exprmat, genes, samples = load_exprmat('../data/expression_matrix.csv')
print('Matrix size:', exprmat.shape)
timepoints = get_timepoints(samples)
projected = reduce_dimensionality(exprmat, 2)

Matrix size: (271, 575)
Explained variance ratio: [ 0.22354609  0.09390967]


To cluster the cells, we will use the k-means algorithm.

## Clustering our gene expression data using k-means

Running k-means is simple. Note that we must tell the algorithm how many clusters we wish to find. In this case, as we hope to see our clusters echo the timepoint assignment of each cell, and because we know there are four timepoints present in our data, we fix the number of clusters at four. Choosing the best cluster number when you lack knowledge *a priori* of the "right" answer is difficult, as we will discuss later.

In [ ]:
import sklearn.cluster
preds = sklearn.cluster.KMeans(n_clusters=4).fit_predict(exprmat)
plot_expression('kmeans', projected, preds, timepoints, samples)

## Clustering via Gaussian mixture models

In [ ]:
import sklearn.mixture
preds = sklearn.mixture.GMM(n_components=4).fit_predict(exprmat)
plot_expression('GMM', projected, preds, timepoints, samples)

## Using Dirichlet process Gaussian mixture models to select the best number of clusters

In [ ]:
preds = sklearn.mixture.DPGMM(n_components=100, alpha=0.3).fit_predict(exprmat)
plot_expression('DPGMM', projected, preds, timepoints, samples)

## Evaluating clustering success

## Choosing the best number of clusters